In [ ]:
"""První část - Příprava datasetu"""

'''dataset
https://www.kaggle.com/iarunava/cell-images-for-detecting-malaria
'''

'''model dává nezávisle na parametrech a na počtu epoch vžy výsledek 50%
Důvodem bude chyba v připravě dat, kterou se mi nepodařilo najít, proto jsem v souboru krev_final.ipynb
zvolil jiný postup řešení a model již funguje'''

In [ ]:
'''
Importy
'''
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2 #pip install opencv-python
from tqdm import tqdm #pip install tqdm


print('načtení dokončeno')

In [ ]:
#cesta ke složce, ve které jsou podsložky s obrázky
path_to_file = "/home/sanctuary/DATAdrive/Documents/archive/cell_images/"
#název podsložek parasitised=nemoc Uninfected=zdravý
files = ["Parasitized", "Uninfected"]
print('cesty načteny')

In [ ]:
"""Ukázka jednoho obrázku malárie a jednoho obrázku zdrávé červenné krvinky"""


for file_ in files:  # foreach pro složky s obrázky
    path = os.path.join(path_to_file,file_) # cesta k jednotlivým složkám

    for img in os.listdir(path):  # foreach pro obrázky ve složce
        array_of_img = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)  # vložení obrázků do pole (array)
        plt.imshow(array_of_img, cmap='gray')  # graf šedá barva
        """^^pravděpodobně budou stačit černobíle obrázky, popřípadě změna zde^^"""
        plt.show()  # ukázat graf
        
        break # break aby tu byl jen jeden obrázek

        
    

In [ ]:
"""je vidět že  každý obrázek má jiný rozměr a je tedy třeba to nějak upravit"""

In [ ]:
new_size = 50
"""změna velikosti obrázků na  new_size * new_size"""
new_array = cv2.resize(array_of_img, (new_size, new_size))
plt.imshow(new_array,cmap='gray' )
plt.show()
# ukázka změny

In [ ]:
print(f'starý rozměr - {array_of_img.shape}')
print(f'nový rozměr - {new_array.shape}') #rozměr obrázku

In [ ]:
#doba trvání je přibližně 2 minuty pro každou složku takže dohromaady asi 4 minuty


training_data = []

#jednotlivé kroky popsány výše


for file_ in files:
    path = os.path.join(path_to_file,file_)   
    index_ = files.index(file_)  # index  0 pro malárii jedna pro zravé 
    for img in tqdm(os.listdir(path)): #foreach pro obrázky    
        try:
            img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)  
            new_array = cv2.resize(img_array, (new_size, new_size))  
            training_data.append([new_array, index_])  # připnutí pole jednoho obrázku do polí s trénovacími daty
        except Exception as e:  # možnost chyby v případě špatné cesty nebo "corrupt" obrázku.
            pass

print('')
print(len(training_data)) #vypíše velikost testovacích dat

In [ ]:
import random
# zamýchání testovacích dat
random.shuffle(training_data)

In [ ]:
#vypíše 0 pokud se jedná o malárii 1 pokud zravou čk
for a in training_data[:20]:
    print(a[1])
    



In [ ]:
features_ = []
labels_ = []

for features,label in training_data: #vložení popisů a feature do samostatných polí
    features_.append(features)
    labels_.append(label)

print(features_[0].reshape(-1, new_size, new_size, 1)) # -1 => počet feature je neomezený, 1 => černobílé (3 pro barevné)

features_ = np.array(features_).reshape(-1, new_size, new_size, 1) # reshape pro použití modulu KERAS

In [ ]:
"""uložení testovacích dat, aby je nebylo nutné vytvářet opakovaně"""

import pickle

pickle_out = open("feature.pickle","wb")
pickle.dump(features_, pickle_out)
pickle_out.close()

pickle_out = open("label.pickle","wb")
pickle.dump(labels_, pickle_out)
pickle_out.close()

print('uloženo')

In [ ]:
"""načtení testovacích dat"""
import pickle

pickle_in = open("feature.pickle","rb")
features_ = pickle.load(pickle_in)

pickle_in = open("label.pickle","rb")
labels_ = pickle.load(pickle_in)

print('načteno')


In [ ]:
"""Druhá část - Model"""

In [ ]:
"""nefunkční model""" 

import numpy as np

import tensorflow as tf #pip install tensorflow

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D


features_=np.array(features_/255.0) #normalizování dat.. scaling
labels_=np.array(labels_)

model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=features_.shape[1:]))# přidat do modelu Convoluční vrstvu [1:]=> přeskočí první sloupec kvůli použití (-1) výše u reshape
                                                                #(3,3) velikost 'pooling window'
                                                                #50 => počet neuronů
model.add(Activation('relu')) 
model.add(MaxPooling2D(pool_size=(2,2),strides=2))


# přidání aktivační vrstvy
model.add(MaxPooling2D(pool_size=(2, 2),strides=2)) # pooling do 2x2

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2),strides=2))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2),strides=2))



model.add(Flatten())  # udělá z modelu jednorozměrný model aby šlo p5idat Dense


model.add(Dense(64,activation='relu'))

# droupout vrstva na regulování (zahodí 20% náhodně vybraných neuronů)
model.add(Dropout(0.2))
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))



model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(features_, labels_, batch_size=100, epochs=10, validation_split=0.1)   

In [ ]:
"""Výsledek modelu

Epoch 1/10
249/249 [==============================] - 10s 38ms/step - loss: 0.6932 - accuracy: 0.5004 - val_loss: 0.6930 - val_accuracy: 0.5134
Epoch 2/10
249/249 [==============================] - 10s 38ms/step - loss: 0.6931 - accuracy: 0.5015 - val_loss: 0.6937 - val_accuracy: 0.4866
Epoch 3/10
249/249 [==============================] - 9s 38ms/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6930 - val_accuracy: 0.5134
Epoch 4/10
249/249 [==============================] - 9s 38ms/step - loss: 0.6932 - accuracy: 0.4977 - val_loss: 0.6932 - val_accuracy: 0.4866
Epoch 5/10
249/249 [==============================] - 9s 37ms/step - loss: 0.6932 - accuracy: 0.4967 - val_loss: 0.6931 - val_accuracy: 0.5134
Epoch 6/10
249/249 [==============================] - 9s 37ms/step - loss: 0.6932 - accuracy: 0.5006 - val_loss: 0.6933 - val_accuracy: 0.4866
Epoch 7/10
249/249 [==============================] - 9s 37ms/step - loss: 0.6932 - accuracy: 0.5015 - val_loss: 0.6933 - val_accuracy: 0.4866
Epoch 8/10
249/249 [==============================] - 9s 37ms/step - loss: 0.6932 - accuracy: 0.5015 - val_loss: 0.6932 - val_accuracy: 0.4866
Epoch 9/10
249/249 [==============================] - 9s 36ms/step - loss: 0.6932 - accuracy: 0.5015 - val_loss: 0.6933 - val_accuracy: 0.4866
Epoch 10/10
249/249 [==============================] - 9s 36ms/step - loss: 0.6932 - accuracy: 0.5015 - val_loss: 0.6933 - val_accuracy: 0.4866

"""